In [0]:
#Importing required libraries
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [0]:
#STEP 1: LOADING DATASET

train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [0]:
#STEP 2: MAKING DATASET ITERABLE

batch_size = 100
n_iters = 6000  
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
len(train_dataset)


60000

In [6]:
num_epochs

10

In [0]:
#STEP 3: CREATE MODEL CLASS

class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        
        # Linear function 1: 784 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim1) 
        # Non-linearity 1
        self.sigmoid = nn.Sigmoid()

        # Linear function 2: 100 --> 100
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        # Non-linearity 2
        self.relu = nn.ReLU()

        # Linear function 3: 100 --> 100
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        # Non-linearity 3
        self.tanh = nn.Tanh()

        # Linear function 4 (readout): 100 --> 10
        self.fc4 = nn.Linear(hidden_dim3, output_dim)  


    def forward(self, x):
        
        out = self.fc1(x)
        # Non-linearity 1
        out = self.sigmoid(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu(out)

        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.tanh(out)

        # Linear function 4 (readout)
        out = self.fc4(out)
        return out


In [0]:
#STEP 4: INSTANTIATE MODEL CLASS

input_dim = 784
hidden_dim1 = 256
hidden_dim2 = 128
hidden_dim3 = 64
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim)


In [0]:
#STEP 5: INSTANTIATE LOSS CLASS

criterion = nn.CrossEntropyLoss()

In [0]:
#STEP 6: INSTANTIATE OPTIMIZER CLASS

learning_rate = 0.1
momentum=0.9

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=momentum)


In [12]:
#STEP 7: TRAIN THE MODEL

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities
                images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = float(100 * correct / total)

            # Print Loss for each Epoch 
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))
            


Iteration: 500. Loss: 0.35782524943351746. Accuracy: 91.0
Iteration: 1000. Loss: 0.19663062691688538. Accuracy: 94.0
Iteration: 1500. Loss: 0.1515016257762909. Accuracy: 95.0
Iteration: 2000. Loss: 0.0488416850566864. Accuracy: 96.0
Iteration: 2500. Loss: 0.12553375959396362. Accuracy: 96.0
Iteration: 3000. Loss: 0.058085847645998. Accuracy: 97.0
Iteration: 3500. Loss: 0.05741754546761513. Accuracy: 97.0
Iteration: 4000. Loss: 0.02063162811100483. Accuracy: 97.0
Iteration: 4500. Loss: 0.0595187172293663. Accuracy: 97.0
Iteration: 5000. Loss: 0.007015925832092762. Accuracy: 97.0
Iteration: 5500. Loss: 0.019600030034780502. Accuracy: 97.0
Iteration: 6000. Loss: 0.06613089889287949. Accuracy: 97.0
